# Dataset analysis: '***2012-13 School Data with Affect***'

Dataset from can be downloaded [here](https://drive.google.com/file/d/0BxCxNjHXlkkHczVDT2kyaTQyZUk/edit).

Description of the dataset can be found on [link1](https://sites.google.com/site/assistmentsdata/home/2012-13-school-data-with-affect) , [link2](https://sites.google.com/site/assistmentsdata/how-to-interpret).

    
## Table of contents
1. Load Dataset   
    1.1 Dimensionality raw dataset
2. Analysis   
    2.1 First attempts   
    2.2 Actions      
    2.3 Attribute 'correct'   
    2.4 Attribute 'problem_type'    
    2.5 Attribute 'original'   
    2.6 Attribute  'template_id' vs 'problem_id': unique problems   
    2.7 Attribute  'template_id' vs 'problem_id': similarity
3. Correctness
    3.1 Correctness per user_id, problem_id   
    3.2 Correctness per original vs scaffolding   
    3.3 Correctness per problem_type


In [1]:
#import libraries,functions
import matplotlib.pylab as pylab
import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None

#set default parameters for plotting
params = {
    'legend.fontsize': 'x-large',
    'figure.figsize': (15, 15),
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'xx-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large',
    'axes.titlepad': 20,
    'axes.titleweight': 500,
    'axes.labelpad': 10
}
pylab.rcParams.update(params)

In [2]:
def percent_correct(data):
    """
    :param a dataframe containing "corrent" column (1 ok, 0 ko)
    :return: percentage of correct answers 
    """
    correct = data[(data.correct == 1)].shape[0]
    wrong = data[(data.correct == 0)].shape[0]

    support = data.shape[0]
    result = correct / (correct + wrong)

    return round(result, 3)


# <font color='blue'>1 Load Dataset</font> 


In [ ]:
#load dataset
df = pd.read_csv(r'../data/ASSISTmentsData_reduced.csv', engine='python')

## 1.1 Dimensionality of the raw dataset

In [ ]:
row, col = df.shape

print("#Action: ", row)
print("#Attribute: ", col)

In [ ]:
print("#unique user: ", df.user_id.nunique())

In [ ]:
print("#unique problem: ", df.problem_id.nunique())

In [ ]:
print("#unique template: ", df.template_id.nunique())

In [ ]:
print("#unique assistment: ", df.assistment_id.nunique())

In [ ]:
#missing values for each attribute

df.isnull().sum(axis=0)

# <font color='blue'>2 Analysis</font> 


In [ ]:
# keep only attributes that we will use
df1 = df[[
    'problem_id', 'user_id', 'problem_type', 'start_time', 'original',
    'correct', 'template_id', 'assistment_id'
]]

## 2.1 First attempt
--------------------------------------------------------------------------
**Consideration**

It is better to consider just the first attempt to a problem, this is done in order to estimate a true  difficulty of the questions in the next part (IRT estimation).      

We are going to drop duplicates 'problem_id' 'user_id', keeping only the first one using the attribute 'start_time'


In [ ]:
#check how many unique problem_id user_id exists = how many first attempts
df1.groupby(['problem_id', 'user_id']).ngroups

In [ ]:
print("#actions before: ", df1.shape[0])

#order rows by start time
df1.sort_values('start_time', inplace=True)

#keep the first duplicate, drop others
df1.drop_duplicates(subset=['problem_id', 'user_id'],
                    keep='first',
                    inplace=True)

print("#actions after: ", df1.shape[0])

## 2.2 Actions


**Description**

A row in the dataset represents an action (i.e the answer of a user_id to a problem_id), each action is identified by a different problem_log_id


--------------------------------------------------------------------------
**Consideration**


In [ ]:
# actions x user
# equivalent: problems x user (since we have only first attempt, pair user_id, problem_id is unique)
action_user = df1.user_id.value_counts()
mean = action_user.mean(axis=0)
std = action_user.std(axis=0)

print("actions x user mean: ", mean)
print("actions x user std: ", std)

In [ ]:
# actions x problem
# equivalent: students x problem 

action_problem = df1.problem_id.value_counts()
mean = action_problem.mean(axis=0)
std = action_problem.std(axis=0)

print("actions x problem mean: ", mean)
print("actions x problem std: ", std)

In [ ]:
# actions x template

action_template = df1.template_id.value_counts()
mean = action_template.mean(axis=0)
std = action_template.std(axis=0)

print("actions x template mean: ", mean)
print("actions x template std: ", std)

In [ ]:
# actions x assistment

action_assistment = df1.assistment_id.value_counts()
mean = action_assistment.mean(axis=0)
std = action_assistment.std(axis=0)

print("actions x assistment mean: ", mean)
print("actions x assistment std: ", std)

In [ ]:
# keep problem/user with interacations greater than a threshold t
# looking at this it's important to see how many problem/user are statistically significant for next step (IRT est)

t = 50
u = action_user[action_user > t].size
p = action_problem[action_problem > t].size

print(u, " users with #actions greater than ", t)
print(p, " problems with #actions greater than ", t)

## 2.X Attribute ***'start_time'***
--------------------------------------------------------------------------
**Description**



--------------------------------------------------------------------------
**Consideration**




In [ ]:
print("min start_time: ",df1.start_time.min())
print("max start_time: ",df1.start_time.max())

In [ ]:
df_time = df1.groupby(['user_id'])['start_time'].agg([('Min' , 'min'), ('Max', 'max')]).add_prefix('Day')

In [ ]:

df_time['DayMax'] = pd.to_datetime(df_time['DayMax'])
df_time['DayMin'] = pd.to_datetime(df_time['DayMin'])


In [ ]:
diff =df_time['DayMax']-df_time['DayMin']

In [ ]:
diff.describe()

In [ ]:
diff.describe()

In [ ]:
# keep only attributes that we will use
temp = df[['problem_id', 'user_id', 'assignment_id']]
temp = temp.groupby('assignment_id').agg('nunique')

In [ ]:
temp.user_id.mean()

In [ ]:
temp.problem_id.mean()

## 2.3 Attribute ***'correct'***
--------------------------------------------------------------------------
**Description**

■ 1 = Correct on first attempt 
  
■ Decimal values are calculated as a partial credit based on the number of hints and attempts needed to solve (based on teacher   setting)  
  
■ 0 = Incorrect on first attempt, or asked for help  

When observed as a dependent variable, it is recommended that this value be converted to a binary variable using the formula: 1 = correct, <1 = Incorrect

--------------------------------------------------------------------------
**Consideration**

In order to have only binary result, round each decimal value as suggested in the ASSISTmentsData website.   
However there are few decimal values in this field and they are associated with 'open_response' problem_type .






In [ ]:
#print counts of unique values attribute 'correct'
df1.correct.value_counts()

In [ ]:
#apply round to 'correct' column. ex: 0.875 --> 0
#cast to int reduce the float to the lower inger
df1['correct'] = df1['correct'].astype(int)

#display counts attribute 'correct' after rounding

x = df1.correct.value_counts(normalize=True).plot.bar()

x.set_title("Answers")
x.set_xlabel("correct")
x.set_ylabel("%");

## 2.4 Attribute ***'problem_type'***
--------------------------------------------------------------------------
**Description**

■ choose_1: Multiple choice (radio buttons)  
■ algebra: Math evaluated string (text box)  
■ fill_in: Simple string-compared answer (text box)  
■ open_response: Records student answer, but their response is always marked correct

--------------------------------------------------------------------------
**Consideration**

Looking at the data this "response is always marked correct" is not true.


In [ ]:
#print counts of unique values attribute 'problem_type'

df1.problem_type.value_counts()

In [ ]:
#look at the distribution of correct attribute in the case of open_response
#As we can see results are not always marked correct, so there is no need to drop these problems

df1[df1.problem_type == 'open_response'].correct.value_counts()

In [ ]:
x = df1.problem_type.value_counts(normalize=True).plot.bar()

x.set_title("Problem_type occurences")
x.set_xlabel("problem_type")
x.set_ylabel("%");

## 2.5 Attribute 'Original'


**Description**

■ 1 = Main problem   
  
■ 0 = Scaffolding problem   

If a problem has scaffolding and the student answers incorrectly or asks for the problem to be broken into steps, a new problem will be created called a scaffolding problem. This creates a separate problem log row in the file with the variable original = 0.

--------------------------------------------------------------------------
**Consideration**

Around 30% problems are scaffolding, in the next section we will analyze the difference in term of correctness 

In [ ]:
x = df1.correct.value_counts(normalize=True).plot.bar()

x.set_title("Original vs Scaffolding")
x.set_xlabel("original")
x.set_ylabel("%");

## 2.6 Attribute 'template_id' vs 'problem_id': unique problems


**Description**

The template ID of the ASSISTment. ASSISTments with the same template ID have similar questions.   

How many problem are unique, not derived from a template? (i.e. how many template has associated only ONE problem_id)  

Example:   Consider this action table

| template_id | problem_id   |
|------|------|
|   12  | 2|
|   12  | 4|
|   3  | 5|
|   3  | 5|
|   10  | 8|
|   10  | 9|


Problem_id 5 is unique, since his template_id appears always associated with problem_id 5   


--------------------------------------------------------------------------
**Consideration**

Around half of the problems do not derive from a template

In [ ]:
back = df1.copy()
back = back[['problem_id', 'template_id']]

In [ ]:
# keep only unique pair problem_id, template_id (keep order doesn't matter)
back = back.drop_duplicates(['problem_id', 'template_id'], keep='last')

In [ ]:
# now a template_id that appears more than 1 times is a template that has associated more than 1 problem_id
f = back.template_id.value_counts().values

# count the how many template_id has associated only 1 problem_id
unique = np.count_nonzero(f == 1)

In [ ]:
print("# total problems: ", df1.problem_id.nunique())
print("# unique problems: ", unique)

## 2.7 Attribute 'template_id' vs 'problem_id':  similarity 

**Description**

We want to verify if problems from the same template presents the almost same correctness.

Procedure:   
    1) keep problem_id with at least 50 actions   (filter)   
    2) calculate correctness associate with each problem_id (group by + aggregate f. mean)   
    3) keep template_id with at least 10 problem_id (filter)   
    4) calculate for each template_id the standard dev. of the problem_id associated with it (should be closest to zero)  

In [ ]:
temp = df1.copy()
temp = temp[['problem_id','template_id','correct']]

In [ ]:
#drop actions associated with problem with less than 50 actions

print("#problem before: ", temp.problem_id.nunique())
print("#action before: ", temp.shape[0])
temp = temp[
    temp['problem_id'].groupby(temp['problem_id']).transform('size') > 50]

print("#problem after: ", temp.problem_id.nunique())
print("#action after: ", temp.shape[0])

In [ ]:
# for each unique problem_id calculate the correctness (that is the mean of the correct value)
temp = temp.groupby('problem_id').agg('mean')

temp.head()

In [ ]:
# drop template_id that has less than 10 problems associated
print("#template before: ", temp.template_id.nunique())

temp = temp[
    temp['template_id'].groupby(temp['template_id']).transform('size') > 10]

print("#template after: ", temp.template_id.nunique())

In [ ]:
# for each unique template_id calculate the standard dev. of the correctness of the problems associated with it
temp = temp.groupby('template_id').agg('std')

temp.head()

In [ ]:
temp.boxplot(column='correct', return_type='axes')



# <font color='blue'>3 Correctness</font> 


## Correctness density
--------------------------------------------------------------------------


Using kernel-density estimate using Gaussian kernels to estimate PDF [link](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.plot.density.html?highlight=density)   
   
X axis: Correctness Range is [0,1]   
Y axis: Density   


Before proceeding, it may be better to remove problem with few actions,
items with low actions could add noise.


--------------------------------------------------------------------------


In [ ]:
#remove problem_id with less than 50 actions
df1 = df1[
    df1['problem_id'].groupby(df1['problem_id']).transform('size') >= 50]

## 3.1 Correctness per user_id, problem_id


In [ ]:
#group by user_id and take the mean of correct answers
x = df1.groupby('user_id').correct.mean().plot.density()

x.set_title("User density correctness")
x.set_xlabel("Correctness")
x.set_xticks([0, 0.25, 0.50, 0.75, 1]);

In [ ]:
#group by problem_id and take the mean of correct answers
x = df1.groupby('problem_id').correct.mean().plot.density()

x.set_title("Problem density correctness")
x.set_xlabel("Correctness")
x.set_xticks([0, 0.25, 0.50, 0.75, 1]);

## 3.2 Correctness per original vs scaffolding

**Consideration**

Scaffolding problem shows a greater difficulty w.r.t main problem.





In [ ]:
print("% correct original: ",
      percent_correct(df1[df1.original == 1]))
print("% correct scaffolding: ",
      percent_correct(df1[df1.original == 0]))

In [ ]:
x = df1[df1.original == 1].groupby('problem_id').correct.mean().plot.density()

x.set_title("Original problem density correctness")
x.set_xlabel("Correctness")
x.set_xticks([0, 0.25, 0.50, 0.75, 1]);

In [ ]:
x = df1[df1.original == 0].groupby('problem_id').correct.mean().plot.density()

x.set_title("Scaffolding problem density correctness")
x.set_xlabel("Correctness")
x.set_xticks([0, 0.25, 0.50, 0.75, 1]);

## 3.3 Correctness per problem_type

**Consideration**

algebra, choose_1, fill_in_1 show similar shapes (they compose 99% of the total number of problems).
open_response, rank, choose_n show very different shapes probably due to the different nature of the problem.




In [ ]:
print("% correct algebra: ",
      percent_correct(df1[df1.problem_type == 'algebra']))
print("% correct choose_1: ",
      percent_correct(df1[df1.problem_type == 'choose_1']))
print("% correct fill_in_1: ",
      percent_correct(df1[df1.problem_type == 'fill_in_1']))
print("% correct open_response : ",
      percent_correct(df[df.problem_type == 'open_response']))
print("% correct choose_n : ",
      percent_correct(df[df.problem_type == 'choose_n']))
print("% correct rank : ",
      percent_correct(df[df.problem_type == 'rank']))

In [ ]:
x = df1[df1.problem_type == 'algebra'].groupby('problem_id').correct.mean().plot.density(bw_method=0.2)

x.set_title("algebra problem density correctness")
x.set_xlabel("Correctness")
x.set_xticks([0, 0.25, 0.50, 0.75, 1]);

In [ ]:
x = df1[df1.problem_type == 'choose_1'].groupby('problem_id').correct.mean().plot.density(bw_method=0.2)

x.set_title("choose_1 problem density correctness")
x.set_xlabel("Correctness")
x.set_xticks([0, 0.25, 0.50, 0.75, 1]);

In [ ]:
x = df1[df1.problem_type == 'fill_in_1'].groupby('problem_id').correct.mean().plot.density(bw_method=0.2)

x.set_title("fill_in_1 problem density correctness")
x.set_xlabel("Correctness")
x.set_xticks([0, 0.25, 0.50, 0.75, 1]);

In [ ]:
x = df1[df1.problem_type == 'open_response'].groupby('problem_id').correct.mean().plot.density(bw_method=0.2)

x.set_title("open_response problem density correctness")
x.set_xlabel("Correctness")
x.set_xticks([0, 0.25, 0.50, 0.75, 1]);

In [ ]:
x = df1[df1.problem_type == 'choose_n'].groupby('problem_id').correct.mean().plot.density(bw_method=0.2)

x.set_title("choose_n problem density correctness")
x.set_xlabel("Correctness")
x.set_xticks([0, 0.25, 0.50, 0.75, 1]);

In [ ]:
x = df1[df1.problem_type == 'rank'].groupby(
    'problem_id').correct.mean().plot.density(bw_method=0.2)

x.set_title("rank problem density correctness")
x.set_xlabel("Correctness")
x.set_xticks([0, 0.25, 0.50, 0.75, 1])